In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from details import folder

In [4]:
folder

'\\\\vector.e3b.columbia.edu\\e3b\\Globus Database\\UWIN Tagged Photos\\\\'

In [5]:
import os

In [6]:
len(os.listdir(os.path.join(folder, r"batch_001")))

13021

In [7]:
data = pd.read_csv(r"\\vector.e3b.columbia.edu\e3b\Globus Database\UWIN Tagged Photos\batch_001\detections_to_update_2024-02-07.csv", index_col=0)

In [8]:
data.head(4)

,photoName,timestamp,speciesID,species,count,siteName,siteAbbrev,longitude,latitude,validationStatus,group,updateCommonName,updateNumIndividuals
1,VID34840-00000.jpg,2023-01-05 17:00:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN
2,VID34840-00001.jpg,2023-01-05 17:30:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN
3,VID34840-00002.jpg,2023-01-06 03:04:45,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN
4,VID34840-00003.jpg,2023-01-06 03:04:46,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN


In [9]:
data["is_present"] = data['species'].apply(lambda x: "not_present" if pd.isna(x) else "present")

In [14]:
data.head(4)

,photoName,timestamp,speciesID,species,count,siteName,siteAbbrev,longitude,latitude,validationStatus,group,updateCommonName,updateNumIndividuals,is_present
1,VID34840-00000.jpg,2023-01-05 17:00:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,not_present
2,VID34840-00001.jpg,2023-01-05 17:30:00,2278,NaN,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,not_present
3,VID34840-00002.jpg,2023-01-06 03:04:45,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,present
4,VID34840-00003.jpg,2023-01-06 03:04:46,14,Eastern cottontail rabbit,1,Alley Pond Park II,ApNo,-73.744904,40.759163,1,1,NaN,NaN,present


## Batch 001

In [15]:
directory = os.path.join(folder, r"batch_001")

In [16]:
datagen = ImageDataGenerator(rescale=1./255.)

In [17]:
training_data = datagen.flow_from_dataframe(dataframe=data, 
                                            directory=directory, 
                                            x_col='photoName', 
                                            y_col='is_present', 
                                            target_size=(224,224), 
                                            class_mode='binary', 
                                            batch_size=32)

Found 14999 validated image filenames belonging to 2 classes.


In [18]:
# validation_data = datagen.flow_from_dataframe(dataframe=data,
#                                               directory=directory,
#                                               x_col='photoName',
#                                               y_col='is_present',
#                                               target_size=(224,224),
#                                               class_mode='binary',
#                                               subset='validation')

In [19]:
from keras.applications import ResNet50
from keras.applications import EfficientNetB4

In [20]:
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [21]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
out = tf.keras.layers.Dense(2, activation='softmax')(x)

In [22]:
base_model.trainable = True

In [23]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = out)

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']      

In [26]:
# callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(filepath='./best_weights_acc_effb4/best_weights_acc.h5', monitor='accuracy', save_best_only=True, mode='max', save_weights_only=True)
# ]

In [ ]:
model.fit(training_data, epochs=4)

Epoch 1/4
162/469 [=========>....................] - ETA: 27:58 - loss: 0.5118 - accuracy: 0.7328

In [ ]:
".\best"